In [1]:
import os

#paper_source_directory = '/home/louis/research/pdf_processor/processed_data/superconductivity_processed/physrevb.71.134526'
#paper_source_directory = '/home/louis/research/pdf_processor/processed_data/superconductivity_processed/physrevb.88.144511'
#paper_source_directory = '/home/louis/research/pdf_processor/processed_data/superconductivity_processed/physrevb.31.1329'
paper_source_directory = '/home/louis/research/pdf_processor/processed_data/superconductivity_processed/physrevb.28.1389'
#paper_source_directory = '/home/louis/research/pdf_processor/processed_data/superconductivity_processed/physrevb.85.214519'
#paper_source_directory = '/home/louis/research/pdf_processor/processed_data/superconductivity_processed/physrevb.98.094505'

file_name = 'text.txt'

with open(os.path.join(paper_source_directory, file_name)) as f:
    paper_text = f.read()


In [2]:
paper_text[:16000]

"Departement de Physique de la Matiere Condensee, Univerxite de Geneve 24 quai Ernest Ansermet, CH-12ll Geneve 4, Switzerlana Lalr,Si, has a high-temperature form crystallizing in the primitive tetragonal CaBe,Ge type structure (space group P4/nmm; a =4.191 A, c=9.944 A), and a low-temperature form of the body-centered tetragonal ThCr,Siz type (space group I4/mmm; a--4.109 A, c-10.299 A). Both structures are BaAl-type derivatives. The high-temperature modifica tion may be quenched from above the transformation temperature of 1720*C and becomes superconducting at 1.6 K, while the low-temperature modification is normal down to 1 K The structures of both modifications have been refined in single-crystal studies. The excep tional polymorphism of Lalr2Si, is discussed, emphasizing structural relationships and the occurrence of superconductivity in the high-temperature modification.\n Over the last decade, the field of superconductivi ty in ternary compounds has received growing interest. Th

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import transformers
import torch


SYS_PROMPT = """You are a condensed matter physicist. You are given the extracted parts of a long document and a question. Read the document and don't make up an answer."""

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# use quantization to lower GPU usage                                                
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=bnb_config
)
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
def format_prompt(prompt, paper_text):
  PROMPT = f"Question: {prompt}\nContext: " + paper_text
  return PROMPT
    \

def generate(formatted_prompt):
  formatted_prompt = formatted_prompt[:4000] # to avoid GPU OOM                      
  messages = [{"role":"system","content":SYS_PROMPT}, {"role":"user","content":formatted_prompt}]
  # tell the model to generate                                                       
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)
  print("sequence length:", input_ids.shape)
  outputs = model.generate(
      input_ids,
      max_new_tokens=1024,
      eos_token_id=terminators,
      do_sample=False, #temperature=0.6, top_p=0.9,
  )
  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [5]:
generate(format_prompt("What is the synthesis method?", paper_text))

/home/louis/miniconda3/envs/llms/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/louis/miniconda3/envs/llms/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


sequence length: torch.Size([1, 1028])


'Based on the provided text, the synthesis method for LaIr2Si is described as follows:\n\n* Preparation of samples: High-purity elements (La, Ir, and Si) were melted together in an argon atmosphere using an arc furnace. Melting losses were less than 0.4%.\n* Homogenization and annealing: The samples were heated above 1000°C in an rf induction furnace in vacuo, and below 1000°C in sealed quartz capsules under argon.\n* Quenching: The high-temperature modification was retained by quenching the samples with an argon jet.\n\nNote that the text does not provide a specific synthesis method for the high-temperature (Lalr2Si-ht) and low-temperature (Lalr2Si-lt) modifications, but rather describes the general procedure used to prepare the samples.'